# Oil Price Prediction

## Constants

In [ ]:
BASE_PATH = 'PATH_TO_THE_BASE_DIRECTORY_IN_GOOGLE_DRIVE'

## Necessary codes

In [ ]:
from google.colab import drive

drive.mount('/MyDrive', force_remount=True)

Mounted at /MyDrive


In [ ]:
!pip install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.2 MB/s eta 0:00:00
time: 268 µs (started: 2024-01-01 20:49:24 +00:00)


## Imports

In [ ]:
!pip install keras-self-attention

  Preparing metadata (setup.py) ... done
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18895 sha256=78eb132fe0b151532b82e02ab9d83d793164e0f46d234ce7404d9717648c8dfd
  Stored in directory: /root/.cache/pip/wheels/b8/f7/24/607b483144fb9c47b4ba2c5fba6b68e54aeee2d5bf6c05302e
Successfully built keras-self-attention
time: 6.13 s (started: 2024-01-01 20:49:24 +00:00)


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import copy
import random
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from keras_self_attention import SeqSelfAttention

time: 3.94 s (started: 2024-01-01 20:49:30 +00:00)


In [ ]:
sns.set()

time: 823 µs (started: 2024-01-01 20:49:34 +00:00)


## Helper Functions

In [ ]:
def flatten(a):
  isChanged = None
  b = []
  while True:
    isChanged = False
    for x in a:
      try:
        b = b + list(x)
        isChanged = True
      except:
        b.append(x)
    a = b
    b = []
    if not isChanged:
      break
  return a

time: 496 µs (started: 2024-01-01 20:49:34 +00:00)


In [ ]:
def show_models_results(model, train_input, train_output, test_input, test_output, squeeze=True):
  def check_and_calc_squeeze(inp):
    if squeeze:
      return np.squeeze(inp)
    else:
      return inp

  print("train and test MAE: ",
        mean_absolute_error(train_output, model.predict(check_and_calc_squeeze(train_input), verbose=0)),
        mean_absolute_error(test_output, model.predict(check_and_calc_squeeze(test_input), verbose=0)))
  print("train and test RMSE: ",
        mean_squared_error(train_output, model.predict(check_and_calc_squeeze(train_input), verbose=0), squared=False),
        mean_squared_error(test_output, model.predict(check_and_calc_squeeze(test_input), verbose=0), squared=False))
  print("train and test MSE: ",
        mean_squared_error(train_output, model.predict(check_and_calc_squeeze(train_input), verbose=0)),
        mean_squared_error(test_output, model.predict(check_and_calc_squeeze(test_input), verbose=0)))
  print("train and test MAPE: ",
        mean_absolute_percentage_error(train_output, model.predict(check_and_calc_squeeze(train_input), verbose=0)),
        mean_absolute_percentage_error(test_output, model.predict(check_and_calc_squeeze(test_input), verbose=0)))

time: 561 µs (started: 2024-01-01 20:49:34 +00:00)


In [ ]:
def show_models_results_with_labels(y_true, y_pred):
  print("MAE: ",
        mean_absolute_error(y_true, y_pred))
  print("RMSE: ",
        mean_squared_error(y_true, y_pred, squared=False))
  print("MSE: ",
        mean_squared_error(y_true, y_pred))
  print("MAPE: ",
        mean_absolute_percentage_error(y_true, y_pred))

time: 553 µs (started: 2024-01-01 20:49:34 +00:00)


In [ ]:
import glob
import pandas as pd
import re
def get_processed_features(datasets_path, suffix):
  """
  inputs:
    suffix -> {daily, weekly, monthly} specifies the interval of features
  output:
    an array of dictionaries, each has 2 keys: {feature, featureName}
  """
  path = datasets_path + "/processed-*-" + suffix + ".csv";
  fileNames = glob.glob(path)
  features = []
  for fileName in fileNames:
    feature = (pd.read_csv(fileName))
    featureName = re.match('.*processed-(.*)-' + suffix + ".csv", fileName).group(1)
    features.append({"feature": feature, "featureName": featureName})
  return features

time: 642 µs (started: 2024-01-01 20:49:34 +00:00)


In [ ]:
def get_dataset(datasets_path, suffix, window_size, start_date, end_date):
  features = get_processed_features(datasets_path, suffix)

  WTI = None
  for feature in features:
    if feature["featureName"] == "WTI-price":
      WTI = feature["feature"]
      break

  WTI = WTI.rename(columns={WTI.columns[1]: "target"})

  prev_wti_target = WTI["target"][:-1]
  prev_wti_target.loc[-1] = 0
  prev_wti_target.index = prev_wti_target.index + 1
  prev_wti_target = prev_wti_target.sort_index()
  WTI["target"] = WTI["target"] - prev_wti_target

  intersection = WTI
  for feature in features:
    df = feature["feature"].rename(columns={feature["feature"].columns[1]: feature["featureName"]})
    intersection = pd.merge(df, intersection, on='Date')

  intersection = (intersection[intersection[intersection.columns[0]] >= start_date])
  intersection = (intersection[intersection[intersection.columns[0]] <= end_date]).reset_index(drop=True)

  return intersection

time: 705 µs (started: 2024-01-01 20:49:34 +00:00)


In [ ]:
import tensorflow as tf
import numpy as np
def get_inputs_and_outputs(df, target, window_size):
  """
  inputs:
    df -> features
    target -> target
    window_size -> window size of inputs
  output:
    [inputs: (m ,window_size, 1), outputs: (m)]
  """
  inputs = np.zeros((len(df[df.columns[0]]) - window_size, window_size, len(df.columns)))
  outputs = np.zeros((len(df[df.columns[0]]) - window_size))

  start = 0
  for i in range(window_size, len(df[df.columns[0]])):
    for idx, feature_column in enumerate(df.columns):
      inputs[start,:,idx] = df[feature_column][start:i]
    outputs[start] = target[i]
    start = start + 1

  inputs = tf.convert_to_tensor(inputs)
  outputs = tf.convert_to_tensor(outputs)

  return [inputs, outputs]

time: 706 µs (started: 2024-01-01 20:49:34 +00:00)


## Data Loading and Preparation

### Constants

In [ ]:
START_DATE = '2008-01-01'
END_DATE = '2023-01-01'
DATASETS_PATH = BASE_PATH + 'data'
TRAIN_PORTION = 0.7
VALIDATION_PORTION = 0.15
TEST_PORTION = 0.15
WINDOW_SIZE = 5

time: 393 µs (started: 2024-01-01 20:49:34 +00:00)


### Load datasets

In [ ]:
daily_dataset = get_dataset(DATASETS_PATH, 'daily', WINDOW_SIZE, START_DATE, END_DATE)
daily_dataset

,Date,dollar-index-2006-2023,propane,jet-fuel,heating-oil,gulf-coast-conventional-gasoline,regular-gasoline,new-york-harbor-conventional-gasoline,WTI-price,Brent-price,platinum-pm-fix,silver-fix,palladium-pm-fix,platinum-am-fix,palladium-am-fix,gold-pm-fix,US-inflation,natural-gas-price,target
0,2008-01-02,89.3514,1.636,2.745,2.739,2.531,2.646,2.544,99.64,97.01,1541.0,14.930,371.0,1530.0,370.0,846.750,2.28,7.83,3.69
1,2008-01-03,89.2310,1.612,2.751,2.718,2.472,2.624,2.509,99.17,98.45,1535.0,15.380,370.0,1546.0,371.0,858.850,2.28,7.84,-0.47
2,2008-01-04,89.2467,1.571,2.716,2.683,2.444,2.586,2.471,97.90,96.87,1545.0,15.275,372.0,1545.0,372.0,855.000,2.25,7.51,-1.27
3,2008-01-07,89.5718,1.538,2.635,2.595,2.371,2.495,2.399,95.08,94.19,1531.0,15.240,370.5,1529.0,367.0,859.250,2.22,7.61,-2.82
4,2008-01-08,89.3398,1.544,2.660,2.622,2.405,2.518,2.436,96.43,96.37,1543.0,15.485,372.0,1541.0,374.5,873.500,2.21,7.59,1.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3658,2022-12-22,122.2105,0.704,2.935,2.957,2.203,2.368,2.363,77.68,79.58,984.0,23.735,1692.0,1005.0,1723.0,1800.700,2.24,7.10,-0.49
3659,2022-12-23,121.9579,0.711,3.049,3.071,2.304,2.529,2.464,79.57,82.45,1000.5,23.735,1743.5,995.0,1658.0,1802.025,2.29,7.15,1.89
3660,2022-12-28,121.9983,0.703,3.308,3.173,2.383,2.517,2.448,78.89,81.70,1017.0,23.855,1795.0,1020.0,1784.0,1803.350,2.33,4.06,-0.56
3661,2022-12-29,121.6556,0.693,3.355,3.150,2.401,2.611,2.474,78.43,80.96,1031.0,23.855,1775.0,1014.0,1775.0,1813.750,2.32,3.78,-0.46


time: 13.7 s (started: 2024-01-01 20:49:34 +00:00)


In [ ]:
weekly_dataset = get_dataset(DATASETS_PATH, 'weekly', WINDOW_SIZE, START_DATE, END_DATE)
weekly_dataset

,Date,Brent-price,WTI-price,imports-products,imports-crude-oil,exports-products,exports-crude-oil,target
0,2008-01-04,96.50,98.17,3098.0,9806.0,1226.0,25,2.53
1,2008-01-11,94.40,94.76,3149.0,10389.0,1212.0,26,-3.41
2,2008-01-18,90.03,91.51,3598.0,10156.0,1212.0,26,-3.25
3,2008-01-25,88.46,89.41,3333.0,10056.0,1212.0,26,-2.10
4,2008-02-01,91.77,91.14,4216.0,10514.0,1329.0,26,1.73
...,...,...,...,...,...,...,...,...
778,2022-12-02,85.03,79.29,2319.0,6012.0,6295.0,3430,0.55
779,2022-12-09,78.56,73.06,2233.0,6867.0,6508.0,4316,-6.23
780,2022-12-16,80.80,75.12,1885.0,5819.0,6344.0,4360,2.06
781,2022-12-23,80.68,77.28,2176.0,6252.0,5756.0,3465,2.16


time: 4.39 s (started: 2024-01-01 20:49:48 +00:00)


In [ ]:
monthly_dataset = get_dataset(DATASETS_PATH, 'monthly', WINDOW_SIZE, START_DATE, END_DATE)
monthly_dataset

,Date,WTI-price,Brent-price,total-renewable-energy-production,total-renewable-energy-consumption,total-primary-energy-consumption,nuclear-electric-power-consumption,total-fossil-fuels-consumption,primary-energy-imports,primary-energy-net-imports,total-primary-energy-production,primary-energy-stock-change-and-other,primary-energy-exports,nuclear-electric-power-production,unemployment,dubai-price,us-oil-production,target
0,2008-01-15,92.97,92.18,0.597514,0.593563,9.410223,0.739318,8.066247,2.948052,2.417403,6.196416,0.796405,0.530649,0.739318,4.8,87.222609,158566,1.28
1,2008-02-15,95.39,94.99,0.542157,0.541614,8.649687,0.680743,7.416912,2.602595,2.079138,5.819765,0.750785,0.523457,0.680743,4.7,90.065238,149297,2.42
2,2008-03-15,105.45,103.64,0.604343,0.596779,8.584242,0.676417,7.303725,2.759004,2.158302,6.163450,0.262490,0.600703,0.676417,4.8,96.373810,160868,10.06
3,2008-04-15,112.58,109.07,0.607143,0.606320,7.832002,0.599242,6.617114,2.775943,2.189734,5.997734,-0.355466,0.586209,0.599242,4.3,103.862727,155403,7.13
4,2008-05-15,125.40,122.80,0.668706,0.664818,7.889757,0.677560,6.539491,2.742533,2.127049,6.214129,-0.451421,0.615485,0.677560,4.6,119.315909,159437,12.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,2022-08-15,93.67,100.45,1.048755,1.041318,8.528718,0.718526,6.748699,1.822905,-0.506708,8.824779,0.210647,2.329613,0.718526,3.5,95.693043,372077,-7.95
176,2022-09-15,84.26,89.76,0.983256,0.965302,7.782286,0.664673,6.139217,1.702808,-0.518462,8.572728,-0.271981,2.221270,0.664673,3.0,89.742727,370120,-9.41
177,2022-10-15,87.55,93.33,1.024145,1.011795,7.810656,0.614741,6.174266,1.768384,-0.555446,8.774752,-0.408650,2.323830,0.614741,3.1,91.214762,384923,3.29
178,2022-11-15,84.37,91.42,1.102058,1.081008,8.283482,0.647029,6.546483,1.762664,-0.544792,8.634988,0.193286,2.307456,0.647029,3.1,84.599091,371378,-3.18


time: 11 s (started: 2024-01-01 20:49:52 +00:00)


### Normalize Datasets

In [ ]:
daily_dataset_just_features = daily_dataset.drop(["Date", "target"], axis=1)
normalized_daily_dataset = (daily_dataset_just_features - daily_dataset_just_features.mean()) / daily_dataset_just_features.std()
normalized_daily_dataset

,dollar-index-2006-2023,propane,jet-fuel,heating-oil,gulf-coast-conventional-gasoline,regular-gasoline,new-york-harbor-conventional-gasoline,WTI-price,Brent-price,platinum-pm-fix,silver-fix,palladium-pm-fix,platinum-am-fix,palladium-am-fix,gold-pm-fix,US-inflation,natural-gas-price
0,-1.291152,2.021658,0.700898,0.676659,0.698149,0.531305,0.614902,1.130897,0.717747,0.976488,-0.865484,-0.967285,0.940894,-0.966672,-1.802632,0.857086,2.032484
1,-1.301646,1.954718,0.708593,0.649506,0.608354,0.499755,0.562151,1.111305,0.771592,0.958606,-0.795949,-0.968750,0.988557,-0.965210,-1.761557,0.857086,2.037632
2,-1.300278,1.840361,0.663709,0.604251,0.565739,0.445260,0.504879,1.058366,0.712512,0.988409,-0.812174,-0.965819,0.985578,-0.963747,-1.774626,0.806568,1.867754
3,-1.271942,1.748318,0.559836,0.490466,0.454636,0.314757,0.396363,0.940817,0.612299,0.946685,-0.817582,-0.968018,0.937915,-0.971058,-1.760199,0.756051,1.919232
4,-1.292163,1.765053,0.591895,0.525377,0.506383,0.347741,0.452128,0.997091,0.693815,0.982449,-0.779724,-0.965819,0.973662,-0.960092,-1.711826,0.739212,1.908936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3658,1.572819,-0.577863,0.944552,0.958533,0.198948,0.132628,0.342105,0.215511,0.065989,-0.683534,0.495093,0.968535,-0.623063,1.011505,1.435657,0.789729,1.656692
3659,1.550803,-0.558339,1.090744,1.105935,0.352665,0.363516,0.494329,0.294294,0.173307,-0.634359,0.495093,1.044004,-0.652853,0.916471,1.440155,0.873925,1.682431
3660,1.554324,-0.580653,1.422882,1.237821,0.472900,0.346307,0.470214,0.265949,0.145262,-0.585184,0.513636,1.119473,-0.578379,1.100691,1.444653,0.941281,0.091749
3661,1.524455,-0.608544,1.483155,1.208082,0.500295,0.481112,0.509401,0.246774,0.117591,-0.543460,0.513636,1.090164,-0.596253,1.087533,1.479956,0.924442,-0.052390


time: 24.2 ms (started: 2024-01-01 20:50:03 +00:00)


In [ ]:
weekly_dataset_just_features = weekly_dataset.drop(["Date", "target"], axis=1)
normalized_weekly_dataset = (weekly_dataset_just_features - weekly_dataset_just_features.mean()) / weekly_dataset_just_features.std()
normalized_weekly_dataset

,Brent-price,WTI-price,imports-products,imports-crude-oil,exports-products,exports-crude-oil
0,0.697527,1.069634,1.803521,1.606827,-1.719152,-0.809675
1,0.619068,0.927393,1.912067,2.071988,-1.728826,-0.808938
2,0.455799,0.791826,2.867706,1.886083,-1.728826,-0.808938
3,0.397141,0.704229,2.303688,1.806296,-1.728826,-0.808938
4,0.520808,0.776392,4.183039,2.171722,-1.647975,-0.808938
...,...,...,...,...,...,...
778,0.268992,0.282094,0.145520,-1.420308,1.783686,1.701009
779,0.027263,0.022222,-0.037520,-0.738125,1.930876,2.354304
780,0.110953,0.108151,-0.778193,-1.574298,1.817546,2.386747
781,0.106470,0.198251,-0.158837,-1.228818,1.411220,1.726817


time: 11.6 ms (started: 2024-01-01 20:50:04 +00:00)


In [ ]:
monthly_dataset_just_features = monthly_dataset.drop(["Date", "target"], axis=1)
normalized_monthly_dataset = (monthly_dataset_just_features - monthly_dataset_just_features.mean()) / monthly_dataset_just_features.std()
normalized_monthly_dataset

,WTI-price,Brent-price,total-renewable-energy-production,total-renewable-energy-consumption,total-primary-energy-consumption,nuclear-electric-power-consumption,total-fossil-fuels-consumption,primary-energy-imports,primary-energy-net-imports,total-primary-energy-production,primary-energy-stock-change-and-other,primary-energy-exports,nuclear-electric-power-production,unemployment,dubai-price,us-oil-production
0,0.858233,0.538657,-1.634860,-1.666612,2.270191,0.927159,2.792208,2.658022,1.916163,-1.197670,1.447016,-1.344608,0.927159,-0.471438,0.457711,-1.347822
1,0.959665,0.644018,-1.996012,-2.017109,0.927995,-0.230019,1.574091,1.550213,1.509696,-1.631195,1.356868,-1.357236,-0.230019,-0.517581,0.569871,-1.467232
2,1.381322,0.968349,-1.590307,-1.644914,0.812498,-0.315481,1.361759,2.051785,1.604822,-1.235614,0.391967,-1.221601,-0.315481,-0.471438,0.818786,-1.318166
3,1.680171,1.171947,-1.572039,-1.580542,-0.515057,-1.840111,0.073715,2.106104,1.642591,-1.426353,-0.829153,-1.247051,-1.840111,-0.702157,1.114272,-1.388570
4,2.217511,1.686753,-1.170399,-1.185859,-0.413131,-0.292901,-0.071902,1.998965,1.567267,-1.177282,-1.018767,-1.195646,-0.292901,-0.563725,1.724001,-1.336601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0.887573,0.848740,1.309064,1.354362,0.714509,0.516403,0.320561,-0.950091,-1.597512,1.827575,0.289521,1.814154,0.516403,-1.071309,0.791925,1.402786
176,0.493160,0.447919,0.881745,0.841487,-0.602796,-0.547490,-0.822793,-1.335217,-1.611636,1.537464,-0.664182,1.623917,-0.547490,-1.302029,0.557146,1.377575
177,0.631058,0.581776,1.148507,1.155172,-0.552729,-1.533921,-0.757043,-1.124929,-1.656077,1.769994,-0.934248,1.804000,-1.533921,-1.255885,0.615228,1.568278
178,0.497771,0.510161,1.656816,1.622148,0.281716,-0.896055,-0.058785,-1.143271,-1.643275,1.609125,0.255215,1.775249,-0.896055,-1.255885,0.354196,1.393781


time: 19.2 ms (started: 2024-01-01 20:50:04 +00:00)


### Prepare inputs and outputs

In [ ]:
daily_inputs, daily_outputs = get_inputs_and_outputs(normalized_daily_dataset, daily_dataset["target"], WINDOW_SIZE)
daily_inputs.shape, daily_outputs.shape, len(normalized_daily_dataset)

(TensorShape([3658, 5, 17]), TensorShape([3658]), 3663)

time: 3.94 s (started: 2024-01-01 20:50:04 +00:00)


In [ ]:
weekly_inputs, weekly_outputs = get_inputs_and_outputs(normalized_weekly_dataset, weekly_dataset["target"], WINDOW_SIZE)
weekly_inputs.shape, weekly_outputs.shape, len(normalized_weekly_dataset)

(TensorShape([778, 5, 6]), TensorShape([778]), 783)

time: 229 ms (started: 2024-01-01 20:50:08 +00:00)


In [ ]:
monthly_inputs, monthly_outputs = get_inputs_and_outputs(normalized_monthly_dataset, monthly_dataset["target"], WINDOW_SIZE)
monthly_inputs.shape, monthly_outputs.shape, len(normalized_monthly_dataset)

(TensorShape([175, 5, 16]), TensorShape([175]), 180)

time: 128 ms (started: 2024-01-01 20:50:08 +00:00)


### Divide into training, validation, and test

In [ ]:
daily_train_inputs, daily_test_inputs, daily_train_outputs, daily_test_outputs = train_test_split(daily_inputs.numpy(), daily_outputs.numpy(), test_size=0.3, shuffle=False)
daily_validation_inputs, daily_test_inputs, daily_validation_outputs, daily_test_outputs = train_test_split(daily_test_inputs, daily_test_outputs, test_size=0.5, shuffle=False)
print(daily_train_inputs.shape, daily_train_outputs.shape)
print(daily_validation_inputs.shape, daily_validation_outputs.shape)
print(daily_test_inputs.shape, daily_test_outputs.shape)

(2560, 5, 17) (2560,)
(549, 5, 17) (549,)
(549, 5, 17) (549,)
time: 5.4 ms (started: 2024-01-01 20:50:08 +00:00)


In [ ]:
weekly_train_inputs, weekly_test_inputs, weekly_train_outputs, weekly_test_outputs = train_test_split(weekly_inputs.numpy(), weekly_outputs.numpy(), test_size=0.3, shuffle=False)
weekly_validation_inputs, weekly_test_inputs, weekly_validation_outputs, weekly_test_outputs = train_test_split(weekly_test_inputs, weekly_test_outputs, test_size=0.5, shuffle=False)
print(weekly_train_inputs.shape, weekly_train_outputs.shape)
print(weekly_validation_inputs.shape, weekly_validation_outputs.shape)
print(weekly_test_inputs.shape, weekly_test_outputs.shape)

(544, 5, 6) (544,)
(117, 5, 6) (117,)
(117, 5, 6) (117,)
time: 2.06 ms (started: 2024-01-01 20:50:08 +00:00)


In [ ]:
monthly_train_inputs, monthly_test_inputs, monthly_train_outputs, monthly_test_outputs = train_test_split(monthly_inputs.numpy(), monthly_outputs.numpy(), test_size=0.3, shuffle=False)
monthly_validation_inputs, monthly_test_inputs, monthly_validation_outputs, monthly_test_outputs = train_test_split(monthly_test_inputs, monthly_test_outputs, test_size=0.5, shuffle=False)
print(monthly_train_inputs.shape, monthly_train_outputs.shape)
print(monthly_validation_inputs.shape, monthly_validation_outputs.shape)
print(monthly_test_inputs.shape, monthly_test_outputs.shape)

(122, 5, 16) (122,)
(26, 5, 16) (26,)
(27, 5, 16) (27,)
time: 2.05 ms (started: 2024-01-01 20:50:08 +00:00)


### Grouping

In [ ]:
train_inputs = [daily_train_inputs, weekly_train_inputs, monthly_train_inputs]
validation_inputs = [daily_validation_inputs, weekly_validation_inputs, monthly_validation_inputs]
test_inputs = [daily_test_inputs, weekly_test_inputs, monthly_test_inputs]

time: 504 µs (started: 2024-01-01 20:50:08 +00:00)


In [ ]:
train_outputs = [daily_train_outputs, weekly_train_outputs, monthly_train_outputs]
validation_outputs = [daily_validation_outputs, weekly_validation_outputs, monthly_validation_outputs]
test_outputs = [daily_test_outputs, weekly_test_outputs, monthly_test_outputs]

time: 319 µs (started: 2024-01-01 20:50:08 +00:00)


## Experiments

### Compile and Fit parameters

In [ ]:
compile_parameters = {
    "optimizer": "adam",
    "loss": "mae"
}

fit_parameters = {
    "batch_size": 64,
    "epochs": 50,
    "shuffle": True,
    "verbose": 0,
}

time: 296 µs (started: 2024-01-01 20:50:08 +00:00)


### Constants

In [ ]:
NUMBER_OF_TRIALS = 5

time: 205 µs (started: 2024-01-01 20:50:08 +00:00)


### Fully Connected Neural Network

#### Model Name

In [ ]:
MODEL_NAME = 'fully_connected'

time: 227 µs (started: 2024-01-01 15:49:06 +00:00)


#### Make the model and fit

In [ ]:
best_wti_models = [None] * 3
best_wti_errors = [[None, None]] * 3
best_wti_histories = [None] * 3

for _ in range(NUMBER_OF_TRIALS):
  daily_model = tf.keras.Sequential([
      tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      tf.keras.layers.Dense(1),
  ])
  weekly_model = tf.keras.models.clone_model(daily_model)
  monthly_model = tf.keras.models.clone_model(daily_model)

  daily_model.compile(**compile_parameters)
  weekly_model.compile(**compile_parameters)
  monthly_model.compile(**compile_parameters)

  daily_model_history = daily_model.fit(
      (daily_train_inputs.reshape(len(daily_train_inputs), -1)),
      daily_train_outputs,
      **fit_parameters,
      validation_data=(
          daily_validation_inputs.reshape(len(daily_validation_inputs), -1),
          daily_validation_outputs)
  )
  weekly_model_history = weekly_model.fit(
      (weekly_train_inputs.reshape(len(weekly_train_inputs), -1)),
      weekly_train_outputs,
      **fit_parameters,
      validation_data=(
          weekly_validation_inputs.reshape(len(weekly_validation_inputs), -1),
          weekly_validation_outputs)
  )
  monthly_model_history = monthly_model.fit(
      (monthly_train_inputs.reshape(len(monthly_train_inputs), -1)),
      monthly_train_outputs,
      **fit_parameters,
      validation_data=(
          monthly_validation_inputs.reshape(len(monthly_validation_inputs), -1),
          monthly_validation_outputs)
  )

  if best_wti_models[0] == None or daily_model_history.history['val_loss'][-1] < best_wti_errors[0][-1]:
    best_wti_models[0] = daily_model
    best_wti_errors[0] = [daily_model_history.history['loss'][-1], daily_model_history.history['val_loss'][-1]]
    best_wti_histories[0] = daily_model_history
  if best_wti_models[1] == None or weekly_model_history.history['val_loss'][-1] < best_wti_errors[1][-1]:
    best_wti_models[1] = weekly_model
    best_wti_errors[1] = [weekly_model_history.history['loss'][-1], weekly_model_history.history['val_loss'][-1]]
    best_wti_histories[1] = weekly_model_history
  if best_wti_models[2] == None or monthly_model_history.history['val_loss'][-1] < best_wti_errors[2][-1]:
    best_wti_models[2] = monthly_model
    best_wti_errors[2] = [monthly_model_history.history['loss'][-1], monthly_model_history.history['val_loss'][-1]]
    best_wti_histories[2] = monthly_model_history

time: 1min 10s (started: 2024-01-01 15:49:06 +00:00)


#### Show Results

In [ ]:
print("Errors: ", best_wti_errors)
print("Test Errors: ")
for i in range(3):
  predicted_labels = best_wti_models[i].predict(
      test_inputs[i].reshape(len(test_inputs[i]), -1),
      verbose=0)
  show_models_results_with_labels(test_outputs[i], predicted_labels)
  print()
  print()

Errors:  [[1.1353158950805664, 1.2113656997680664], [1.533952236175537, 2.8138656616210938], [1.2398581504821777, 5.499279499053955]]
Test Errors: 
MAE:  1.7757285427033578
RMSE:  2.5953366725113636
MSE:  6.7357724436823565
MAPE:  5897788069477.29


MAE:  4.5734891768029104
RMSE:  5.567469330366718
MSE:  30.996714744574028
MAPE:  7.726813674376177


MAE:  6.168829714302665
RMSE:  7.652201838816393
MSE:  58.55619298198499
MAPE:  2.2377034201126325


time: 414 ms (started: 2024-01-01 15:50:16 +00:00)


### CNN

#### Model Name

In [ ]:
MODEL_NAME = 'CNN'

time: 333 µs (started: 2024-01-01 15:50:18 +00:00)


#### Make the model and fit

In [ ]:
best_wti_models = [None] * 3
best_wti_errors = [[None, None]] * 3
best_wti_histories = [None] * 3

for _ in range(NUMBER_OF_TRIALS):
  daily_model = tf.keras.Sequential([
      tf.keras.layers.Conv1D(64, 1, input_shape=(daily_train_inputs.shape[1:])),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      tf.keras.layers.Dense(1),
  ])
  weekly_model = tf.keras.Sequential([
      tf.keras.layers.Conv1D(64, 1, input_shape=(weekly_train_inputs.shape[1:])),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      tf.keras.layers.Dense(1),
  ])
  monthly_model = tf.keras.Sequential([
      tf.keras.layers.Conv1D(64, 1, input_shape=(monthly_train_inputs.shape[1:])),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      tf.keras.layers.Dense(1),
  ])

  daily_model.compile(**compile_parameters)
  weekly_model.compile(**compile_parameters)
  monthly_model.compile(**compile_parameters)

  daily_model_history = daily_model.fit(
      (daily_train_inputs),
      daily_train_outputs,
      **fit_parameters,
      validation_data=(
          daily_validation_inputs,
          daily_validation_outputs)
  )
  weekly_model_history = weekly_model.fit(
      (weekly_train_inputs),
      weekly_train_outputs,
      **fit_parameters,
      validation_data=(
          weekly_validation_inputs,
          weekly_validation_outputs)
  )
  monthly_model_history = monthly_model.fit(
      (monthly_train_inputs),
      monthly_train_outputs,
      **fit_parameters,
      validation_data=(
          monthly_validation_inputs,
          monthly_validation_outputs)
  )

  if best_wti_models[0] == None or daily_model_history.history['val_loss'][-1] < best_wti_errors[0][-1]:
    best_wti_models[0] = daily_model
    best_wti_errors[0] = [daily_model_history.history['loss'][-1], daily_model_history.history['val_loss'][-1]]
    best_wti_histories[0] = daily_model_history
  if best_wti_models[1] == None or weekly_model_history.history['val_loss'][-1] < best_wti_errors[1][-1]:
    best_wti_models[1] = weekly_model
    best_wti_errors[1] = [weekly_model_history.history['loss'][-1], weekly_model_history.history['val_loss'][-1]]
    best_wti_histories[1] = weekly_model_history
  if best_wti_models[2] == None or monthly_model_history.history['val_loss'][-1] < best_wti_errors[2][-1]:
    best_wti_models[2] = monthly_model
    best_wti_errors[2] = [monthly_model_history.history['loss'][-1], monthly_model_history.history['val_loss'][-1]]
    best_wti_histories[2] = monthly_model_history

time: 1min 22s (started: 2024-01-01 15:50:18 +00:00)


#### Show Results

In [ ]:
print("Errors: ", best_wti_errors)
print("Test Errors: ")
for i in range(3):
  predicted_labels = best_wti_models[i].predict(
      (test_inputs[i]),
      verbose=0)
  show_models_results_with_labels(test_outputs[i], predicted_labels)
  print()
  print()

Errors:  [[1.128460168838501, 1.2164198160171509], [1.4296574592590332, 2.843658447265625], [0.9168164730072021, 5.4821624755859375]]
Test Errors: 
MAE:  1.7705841704960132
RMSE:  2.4294784614763203
MSE:  5.902365594777349
MAPE:  2737364450644.6694


MAE:  3.7356536508090477
RMSE:  5.04614450037068
MSE:  25.463574318621255
MAPE:  3.761295142535048


MAE:  6.139663217310551
RMSE:  7.5100452008723595
MSE:  56.400778919145964
MAPE:  2.916122741671749


time: 543 ms (started: 2024-01-01 15:51:41 +00:00)


### CNN-LSTM

#### Model Name

In [ ]:
MODEL_NAME = 'CNN-LSTM'

time: 486 µs (started: 2024-01-01 15:52:04 +00:00)


#### Make the model and fit

In [ ]:
best_wti_models = [None] * 3
best_wti_errors = [[None, None]] * 3
best_wti_histories = [None] * 3

for _ in range(NUMBER_OF_TRIALS):
  daily_model = tf.keras.Sequential([
      tf.keras.layers.Conv1D(64, 1, input_shape=(daily_train_inputs.shape[1:])),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Reshape((-1, 1)),
      tf.keras.layers.LSTM(64),
      # tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      # tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      # tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      tf.keras.layers.Dense(1),
  ])
  weekly_model = tf.keras.Sequential([
      tf.keras.layers.Conv1D(64, 1, input_shape=(weekly_train_inputs.shape[1:])),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Reshape((-1, 1)),
      tf.keras.layers.LSTM(64),
      # tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      # tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      # tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      tf.keras.layers.Dense(1),
  ])
  monthly_model = tf.keras.Sequential([
      tf.keras.layers.Conv1D(64, 1, input_shape=(monthly_train_inputs.shape[1:])),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Reshape((-1, 1)),
      tf.keras.layers.LSTM(64),
      # tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      # tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      # tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      tf.keras.layers.Dense(1),
  ])

  daily_model.compile(**compile_parameters)
  weekly_model.compile(**compile_parameters)
  monthly_model.compile(**compile_parameters)

  daily_model_history = daily_model.fit(
      (daily_train_inputs),
      daily_train_outputs,
      **fit_parameters,
      validation_data=(
          daily_validation_inputs,
          daily_validation_outputs)
  )
  weekly_model_history = weekly_model.fit(
      (weekly_train_inputs),
      weekly_train_outputs,
      **fit_parameters,
      validation_data=(
          weekly_validation_inputs,
          weekly_validation_outputs)
  )
  monthly_model_history = monthly_model.fit(
      (monthly_train_inputs),
      monthly_train_outputs,
      **fit_parameters,
      validation_data=(
          monthly_validation_inputs,
          monthly_validation_outputs)
  )

  if best_wti_models[0] == None or daily_model_history.history['val_loss'][-1] < best_wti_errors[0][-1]:
    best_wti_models[0] = daily_model
    best_wti_errors[0] = [daily_model_history.history['loss'][-1], daily_model_history.history['val_loss'][-1]]
    best_wti_histories[0] = daily_model_history
  if best_wti_models[1] == None or weekly_model_history.history['val_loss'][-1] < best_wti_errors[1][-1]:
    best_wti_models[1] = weekly_model
    best_wti_errors[1] = [weekly_model_history.history['loss'][-1], weekly_model_history.history['val_loss'][-1]]
    best_wti_histories[1] = weekly_model_history
  if best_wti_models[2] == None or monthly_model_history.history['val_loss'][-1] < best_wti_errors[2][-1]:
    best_wti_models[2] = monthly_model
    best_wti_errors[2] = [monthly_model_history.history['loss'][-1], monthly_model_history.history['val_loss'][-1]]
    best_wti_histories[2] = monthly_model_history

time: 3min 12s (started: 2024-01-01 15:53:53 +00:00)


#### Show Results

In [ ]:
print("Errors: ", best_wti_errors)
print("Test Errors: ")
for i in range(3):
  predicted_labels = best_wti_models[i].predict(
      (test_inputs[i]),
      verbose=0)
  show_models_results_with_labels(test_outputs[i], predicted_labels)
  print()
  print()

Errors:  [[1.177356243133545, 1.186719536781311], [2.1522274017333984, 2.104658842086792], [4.766221046447754, 5.12949800491333]]
Test Errors: 
MAE:  1.56348698613965
RMSE:  2.2678688406834757
MSE:  5.143229078543012
MAPE:  3162859340553.556


MAE:  2.7766492873360193
RMSE:  3.8641838421068555
MSE:  14.9319167655997
MAPE:  1.0690482732724071


MAE:  6.089693772836967
RMSE:  7.3993379055152095
MSE:  54.75020143999421
MAPE:  1.3291345994299117


time: 1.29 s (started: 2024-01-01 15:57:05 +00:00)


### CNN-LSTM-Attention

#### Model Name

In [ ]:
MODEL_NAME = 'CNN-LSTM-AM'

time: 226 µs (started: 2024-01-01 20:50:08 +00:00)


#### Make the model and fit

In [ ]:
from keras.layers import Layer
import tensorflow as tf
import keras.backend as K
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

time: 812 µs (started: 2024-01-01 20:50:08 +00:00)


In [ ]:
best_wti_models = [None] * 3
best_wti_errors = [[None, None]] * 3
best_wti_histories = [None] * 3

for _ in range(NUMBER_OF_TRIALS):
  daily_model = tf.keras.Sequential([
      tf.keras.layers.Conv1D(64, 1, input_shape=(daily_train_inputs.shape[1:])),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Reshape((-1, 1)),
      tf.keras.layers.LSTM(64, return_sequences=True),
      attention(),
      tf.keras.layers.Dense(1),
  ])
  weekly_model = tf.keras.Sequential([
      tf.keras.layers.Conv1D(64, 1, input_shape=(weekly_train_inputs.shape[1:])),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Reshape((-1, 1)),
      tf.keras.layers.LSTM(64, return_sequences=True),
      attention(),
      tf.keras.layers.Dense(1),
  ])
  monthly_model = tf.keras.Sequential([
      tf.keras.layers.Conv1D(64, 1, input_shape=(monthly_train_inputs.shape[1:])),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Reshape((-1, 1)),
      tf.keras.layers.LSTM(64, return_sequences=True),
      attention(),
      tf.keras.layers.Dense(1),
  ])

  daily_model.compile(**compile_parameters)
  weekly_model.compile(**compile_parameters)
  monthly_model.compile(**compile_parameters)

  daily_model_history = daily_model.fit(
      (daily_train_inputs),
      daily_train_outputs,
      **fit_parameters,
      validation_data=(
          daily_validation_inputs,
          daily_validation_outputs)
  )
  weekly_model_history = weekly_model.fit(
      (weekly_train_inputs),
      weekly_train_outputs,
      **fit_parameters,
      validation_data=(
          weekly_validation_inputs,
          weekly_validation_outputs)
  )
  monthly_model_history = monthly_model.fit(
      (monthly_train_inputs),
      monthly_train_outputs,
      **fit_parameters,
      validation_data=(
          monthly_validation_inputs,
          monthly_validation_outputs)
  )

  if best_wti_models[0] == None or daily_model_history.history['val_loss'][-1] < best_wti_errors[0][-1]:
    best_wti_models[0] = daily_model
    best_wti_errors[0] = [daily_model_history.history['loss'][-1], daily_model_history.history['val_loss'][-1]]
    best_wti_histories[0] = daily_model_history
  if best_wti_models[1] == None or weekly_model_history.history['val_loss'][-1] < best_wti_errors[1][-1]:
    best_wti_models[1] = weekly_model
    best_wti_errors[1] = [weekly_model_history.history['loss'][-1], weekly_model_history.history['val_loss'][-1]]
    best_wti_histories[1] = weekly_model_history
  if best_wti_models[2] == None or monthly_model_history.history['val_loss'][-1] < best_wti_errors[2][-1]:
    best_wti_models[2] = monthly_model
    best_wti_errors[2] = [monthly_model_history.history['loss'][-1], monthly_model_history.history['val_loss'][-1]]
    best_wti_histories[2] = monthly_model_history

time: 3min 28s (started: 2024-01-01 20:50:09 +00:00)


#### Show Results

In [ ]:
print("Errors: ", best_wti_errors)
print("Test Errors: ")
for i in range(3):
  predicted_labels = best_wti_models[i].predict(
      (test_inputs[i]),
      verbose=0)
  show_models_results_with_labels(test_outputs[i], predicted_labels)
  print()
  print()

Errors:  [[1.175446629524231, 1.1915802955627441], [2.1400506496429443, 2.104034423828125], [4.236766338348389, 5.247283935546875]]
Test Errors: 
MAE:  1.5557936891323658
RMSE:  2.242274169435917
MSE:  5.027793450919531
MAPE:  1508935350500.9456


MAE:  2.7755088150400944
RMSE:  3.8623708006891913
MSE:  14.917908202016465
MAPE:  1.058214964974519


MAE:  5.6020090616190865
RMSE:  6.708890878094102
MSE:  45.00921681417425
MAPE:  1.6490190527067665


time: 1.37 s (started: 2024-01-01 16:19:56 +00:00)


### CNN-BiLSTM

#### Model Name

In [ ]:
MODEL_NAME = 'CNN-BiLSTM'

time: 371 µs (started: 2024-01-01 16:10:02 +00:00)


#### Make the model and fit

In [ ]:
best_wti_models = [None] * 3
best_wti_errors = [[None, None]] * 3
best_wti_histories = [None] * 3

for _ in range(NUMBER_OF_TRIALS):
  daily_model = tf.keras.Sequential([
      tf.keras.layers.Conv1D(64, 1, input_shape=(daily_train_inputs.shape[1:])),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Reshape((-1, 1)),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
      tf.keras.layers.Dense(1),
  ])
  weekly_model = tf.keras.Sequential([
      tf.keras.layers.Conv1D(64, 1, input_shape=(weekly_train_inputs.shape[1:])),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Reshape((-1, 1)),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
      tf.keras.layers.Dense(1),
  ])
  monthly_model = tf.keras.Sequential([
      tf.keras.layers.Conv1D(64, 1, input_shape=(monthly_train_inputs.shape[1:])),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Reshape((-1, 1)),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
      tf.keras.layers.Dense(1),
  ])

  daily_model.compile(**compile_parameters)
  weekly_model.compile(**compile_parameters)
  monthly_model.compile(**compile_parameters)

  daily_model_history = daily_model.fit(
      (daily_train_inputs),
      daily_train_outputs,
      **fit_parameters,
      validation_data=(
          daily_validation_inputs,
          daily_validation_outputs)
  )
  weekly_model_history = weekly_model.fit(
      (weekly_train_inputs),
      weekly_train_outputs,
      **fit_parameters,
      validation_data=(
          weekly_validation_inputs,
          weekly_validation_outputs)
  )
  monthly_model_history = monthly_model.fit(
      (monthly_train_inputs),
      monthly_train_outputs,
      **fit_parameters,
      validation_data=(
          monthly_validation_inputs,
          monthly_validation_outputs)
  )

  if best_wti_models[0] == None or daily_model_history.history['val_loss'][-1] < best_wti_errors[0][-1]:
    best_wti_models[0] = daily_model
    best_wti_errors[0] = [daily_model_history.history['loss'][-1], daily_model_history.history['val_loss'][-1]]
    best_wti_histories[0] = daily_model_history
  if best_wti_models[1] == None or weekly_model_history.history['val_loss'][-1] < best_wti_errors[1][-1]:
    best_wti_models[1] = weekly_model
    best_wti_errors[1] = [weekly_model_history.history['loss'][-1], weekly_model_history.history['val_loss'][-1]]
    best_wti_histories[1] = weekly_model_history
  if best_wti_models[2] == None or monthly_model_history.history['val_loss'][-1] < best_wti_errors[2][-1]:
    best_wti_models[2] = monthly_model
    best_wti_errors[2] = [monthly_model_history.history['loss'][-1], monthly_model_history.history['val_loss'][-1]]
    best_wti_histories[2] = monthly_model_history

time: 5min 31s (started: 2024-01-01 16:10:02 +00:00)


#### Show Results

In [ ]:
print("Errors: ", best_wti_errors)
print("Test Errors: ")
for i in range(3):
  predicted_labels = best_wti_models[i].predict(
      (test_inputs[i]),
      verbose=0)
  show_models_results_with_labels(test_outputs[i], predicted_labels)
  print()
  print()

Errors:  [[1.170648455619812, 1.2023967504501343], [2.1339282989501953, 2.1180686950683594], [4.408439636230469, 4.919065475463867]]
Test Errors: 
MAE:  1.5898362184889978
RMSE:  2.27380670733388
MSE:  5.17019694231654
MAPE:  2634211403565.2603


MAE:  2.7469349652936312
RMSE:  3.863443107866213
MSE:  14.926192647718942
MAPE:  1.5199377484456553


MAE:  6.568306853583566
RMSE:  7.9407595683563095
MSE:  63.05566252244228
MAPE:  1.3878854280982307


time: 2.15 s (started: 2024-01-01 16:15:34 +00:00)


### CNN-LSTM-DNN

#### Model Name

In [ ]:
MODEL_NAME = 'CNN-LSTM-DNN'

time: 417 µs (started: 2024-01-01 16:06:19 +00:00)


#### Make the model and fit

In [ ]:
best_wti_models = [None] * 3
best_wti_errors = [[None, None]] * 3
best_wti_histories = [None] * 3

for _ in range(NUMBER_OF_TRIALS):
  daily_model = tf.keras.Sequential([
      tf.keras.layers.Conv1D(64, 1, input_shape=(daily_train_inputs.shape[1:])),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Reshape((-1, 1)),
      tf.keras.layers.LSTM(64),
      tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      tf.keras.layers.Dense(1),
  ])
  weekly_model = tf.keras.Sequential([
      tf.keras.layers.Conv1D(64, 1, input_shape=(weekly_train_inputs.shape[1:])),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Reshape((-1, 1)),
      tf.keras.layers.LSTM(64),
      tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      tf.keras.layers.Dense(1),
  ])
  monthly_model = tf.keras.Sequential([
      tf.keras.layers.Conv1D(64, 1, input_shape=(monthly_train_inputs.shape[1:])),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Reshape((-1, 1)),
      tf.keras.layers.LSTM(64),
      tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      tf.keras.layers.Dense(random.randint(32, 512), 'leaky_relu'),
      tf.keras.layers.Dense(1),
  ])

  daily_model.compile(**compile_parameters)
  weekly_model.compile(**compile_parameters)
  monthly_model.compile(**compile_parameters)

  daily_model_history = daily_model.fit(
      (daily_train_inputs),
      daily_train_outputs,
      **fit_parameters,
      validation_data=(
          daily_validation_inputs,
          daily_validation_outputs)
  )
  weekly_model_history = weekly_model.fit(
      (weekly_train_inputs),
      weekly_train_outputs,
      **fit_parameters,
      validation_data=(
          weekly_validation_inputs,
          weekly_validation_outputs)
  )
  monthly_model_history = monthly_model.fit(
      (monthly_train_inputs),
      monthly_train_outputs,
      **fit_parameters,
      validation_data=(
          monthly_validation_inputs,
          monthly_validation_outputs)
  )

  if best_wti_models[0] == None or daily_model_history.history['val_loss'][-1] < best_wti_errors[0][-1]:
    best_wti_models[0] = daily_model
    best_wti_errors[0] = [daily_model_history.history['loss'][-1], daily_model_history.history['val_loss'][-1]]
    best_wti_histories[0] = daily_model_history
  if best_wti_models[1] == None or weekly_model_history.history['val_loss'][-1] < best_wti_errors[1][-1]:
    best_wti_models[1] = weekly_model
    best_wti_errors[1] = [weekly_model_history.history['loss'][-1], weekly_model_history.history['val_loss'][-1]]
    best_wti_histories[1] = weekly_model_history
  if best_wti_models[2] == None or monthly_model_history.history['val_loss'][-1] < best_wti_errors[2][-1]:
    best_wti_models[2] = monthly_model
    best_wti_errors[2] = [monthly_model_history.history['loss'][-1], monthly_model_history.history['val_loss'][-1]]
    best_wti_histories[2] = monthly_model_history

time: 3min 29s (started: 2024-01-01 16:06:19 +00:00)


#### Show Results

In [ ]:
print("Errors: ", best_wti_errors)
print("Test Errors: ")
for i in range(3):
  predicted_labels = best_wti_models[i].predict(
      (test_inputs[i]),
      verbose=0)
  show_models_results_with_labels(test_outputs[i], predicted_labels)
  print()
  print()

Errors:  [[1.1738712787628174, 1.1832911968231201], [2.1466307640075684, 2.2550716400146484], [4.1481122970581055, 5.079004287719727]]
Test Errors: 
MAE:  1.5534507647551632
RMSE:  2.2429818509383903
MSE:  5.030967583639008
MAPE:  2694630650640.8354


MAE:  2.6477320867468013
RMSE:  3.8170305576852464
MSE:  14.569722278302942
MAPE:  1.8984538421390063


MAE:  5.915256439800616
RMSE:  7.05974269632288
MSE:  49.839966938284256
MAPE:  1.6525688549326332


time: 1.36 s (started: 2024-01-01 16:09:49 +00:00)
